In [1]:
# install necessary packages

!pip3 install psycopg2-binary
!pip3 install pandas

# import necessary packages

import pandas as pd
import psycopg2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
# connect to the DB using psycopg2

db_host = "codd01.research.northwestern.edu"
db_user = "cpdbstudent"
db_password = "DataSci4AI"
db_name = "postgres"

connection = psycopg2.connect(host = db_host,  
                              user = db_user, 
                              password = db_password, 
                              dbname = db_name)

connection

<connection object at 0x168ce02e0; dsn: 'user=cpdbstudent password=xxx dbname=postgres host=codd01.research.northwestern.edu', closed: 0>

## Sentiment Analysis for Complaint Reports Against Officers with >= 90th TRR Percentile

In [3]:
# query from the DB and store as data frame - complaint reports for officers with >= 90th TRR percentile

query_90 = "SELECT crid as cr_id, cr_text FROM data_allegation a JOIN data_officerallegation d on a.crid = d.allegation_id JOIN data_officer o ON d.officer_id = o.id WHERE trr_percentile >= 90 and ((cr_text <> '' AND cr_text IS NOT NULL) OR summary <> '');"
df_90 = pd.read_sql(query_90, con=connection)

display(df_90)

,cr_id,cr_text
0,1045912,None
1,1082479,None
2,1081379,None
3,1081602,None
4,1081684,None
...,...,...
2332,1003683,None
2333,1004549,None
2334,1056371,Initial / Intake Allegation 1: The reporting p...
2335,1058913,Initial / Intake Allegation 1: It is reported ...


In [4]:
# split the cr_text column into a column name and text body

df_90[['column_name', 'text']] = df_90['cr_text'].str.split(":", 1, expand = True)

display(df_90)

,cr_id,cr_text,column_name,text
0,1045912,None,None,None
1,1082479,None,None,None
2,1081379,None,None,None
3,1081602,None,None,None
4,1081684,None,None,None
...,...,...,...,...
2332,1003683,None,None,None
2333,1004549,None,None,None
2334,1056371,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party further alleged that\nwhi...
2335,1058913,Initial / Intake Allegation 1: It is reported ...,Initial / Intake Allegation 1,It is reported that while the accused was.\ni...


In [5]:
# remove rows where column name is not 'Initial / Intake Allegation' or 'Allegation'

df_90 = df_90[df_90['column_name'].isin(['Initial / Intake Allegation 1', 'Allegation 1'])]

display(df_90)

,cr_id,cr_text,column_name,text
16,1058889,Initial / Intake Allegation 1: party\nto go to...,Initial / Intake Allegation 1,"party\nto go to court on\nof radio\nagencey,\..."
101,1049135,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the accused\...
102,1050472,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the\naccused...
103,1050934,Initial / Intake Allegation 1: ACCUSED\nGARCIA...,Initial / Intake Allegation 1,"ACCUSED\nGARCIA\n6490\n008/ -\n, LUIS\nébs/OS..."
104,1052558,Initial / Intake Allegation 1: It is reported ...,Initial / Intake Allegation 1,It is reported that the accused violated a\ns...
...,...,...,...,...
2185,1054894,Initial / Intake Allegation 1: Victim #2 ~~~) ...,Initial / Intake Allegation 1,Victim #2 ~~~) alleges that he was\nforced of...
2323,1054482,Initial / Intake Allegation 1: THE REPORTING P...,Initial / Intake Allegation 1,THE REPORTING PARTY STATES THAT\nTHE ACCUSED ...
2334,1056371,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party further alleged that\nwhi...
2335,1058913,Initial / Intake Allegation 1: It is reported ...,Initial / Intake Allegation 1,It is reported that while the accused was.\ni...


In [6]:
# remove return and new line characters 

df_90['text'] = df_90['text'].str.replace('\\n', ' ')
df_90['text'] = df_90['text'].str.replace('\\r', ' ')

# remove extra whitespace

df_90['text'] = df_90['text'].str.strip()

display(df_90)

/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/1897399502.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_90['text'] = df_90['text'].str.replace('\\n', ' ')
/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/1897399502.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_90['text'] = df_90['text'].str.replace('\\r', ' ')


,cr_id,cr_text,column_name,text
16,1058889,Initial / Intake Allegation 1: party\nto go to...,Initial / Intake Allegation 1,"party to go to court on of radio agencey, Depa..."
101,1049135,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the accused o...
102,1050472,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the accused o...
103,1050934,Initial / Intake Allegation 1: ACCUSED\nGARCIA...,Initial / Intake Allegation 1,"ACCUSED GARCIA 6490 008/ - , LUIS ébs/OSO 93¢ ..."
104,1052558,Initial / Intake Allegation 1: It is reported ...,Initial / Intake Allegation 1,It is reported that the accused violated a sta...
...,...,...,...,...
2185,1054894,Initial / Intake Allegation 1: Victim #2 ~~~) ...,Initial / Intake Allegation 1,Victim #2 ~~~) alleges that he was forced off ...
2323,1054482,Initial / Intake Allegation 1: THE REPORTING P...,Initial / Intake Allegation 1,THE REPORTING PARTY STATES THAT THE ACCUSED OF...
2334,1056371,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party further alleged that while...
2335,1058913,Initial / Intake Allegation 1: It is reported ...,Initial / Intake Allegation 1,It is reported that while the accused was. ins...


In [7]:
# drop unnecessary columns

df_90 = df_90.drop(['cr_text', 'column_name'], axis=1)

display(df_90)

,cr_id,text
16,1058889,"party to go to court on of radio agencey, Depa..."
101,1049135,The reporting party alleges that the accused o...
102,1050472,The reporting party alleges that the accused o...
103,1050934,"ACCUSED GARCIA 6490 008/ - , LUIS ébs/OSO 93¢ ..."
104,1052558,It is reported that the accused violated a sta...
...,...,...
2185,1054894,Victim #2 ~~~) alleges that he was forced off ...
2323,1054482,THE REPORTING PARTY STATES THAT THE ACCUSED OF...
2334,1056371,The reporting party further alleged that while...
2335,1058913,It is reported that while the accused was. ins...


In [8]:
# remove duplicates

df_90 = df_90.sort_values('text').drop_duplicates('cr_id', keep='last')
df_90 = df_90.sort_values('cr_id')

display(df_90)

,cr_id,text
262,1048978,The reporting party alleges that several unifo...
880,1048979,The complainant alleges that the accused appro...
902,1048996,The reporting party alleges that during his co...
905,1049004,The reporting party alleges that the accused f...
907,1049008,The reporting party alleges that the accused o...
...,...,...
2095,1075935,The complainant alleged that the accused detai...
2108,1076007,The reporting victim alleged the accused direc...
2111,1076026,The reporting victim alleges that the accused ...
2114,1076159,The reporting party/victim alleg t the accused...


In [9]:
# make all text lowercase

df_90['text'] = df_90['text'].str.lower()

display(df_90)

,cr_id,text
262,1048978,the reporting party alleges that several unifo...
880,1048979,the complainant alleges that the accused appro...
902,1048996,the reporting party alleges that during his co...
905,1049004,the reporting party alleges that the accused f...
907,1049008,the reporting party alleges that the accused o...
...,...,...
2095,1075935,the complainant alleged that the accused detai...
2108,1076007,the reporting victim alleged the accused direc...
2111,1076026,the reporting victim alleges that the accused ...
2114,1076159,the reporting party/victim alleg t the accused...


In [10]:
# install NLTK
!pip3 install --user -U nltk

# import NLTK and sentiment analyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [11]:
# initialize the sentiment analyzer

nltk.download('vader_lexicon')
sentiment_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/madisonmcclellan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [12]:
# analyze positive and negative sentiment

sentiments_90 = []
positive_90 = 0
negative_90 = 0
neutral_90 = 0

for i in range(df_90.shape[0]):
    scores = sentiment_analyzer.polarity_scores(df_90.iloc[i]['text'])
    sentiments_90.append(scores["compound"])
    if scores["compound"] > 0:
        positive_90 += 1
    elif scores["compound"] < 0:
        negative_90 += 1
    else:
        neutral_90 += 1
        
print("Total reports: ", len(sentiments_90))
print("Number of reports with positive sentiment: ", positive_90)
print("Number of reports with negative sentiment: ", negative_90)
print("Number of reports with neutral sentiment: ", neutral_90)

Total reports:  1283
Number of reports with positive sentiment:  136
Number of reports with negative sentiment:  1139
Number of reports with neutral sentiment:  8


In [13]:
# calculate the proportion of positive and negative sentiments

proportion_pos_90 = positive_90 / len(sentiments_90)
print("Proportion of reports with positive sentiment: ", proportion_pos_90)

proportion_neg_90 = negative_90 / len(sentiments_90)
print("Proportion of reports with negative sentiment: ", proportion_neg_90)

Proportion of reports with positive sentiment:  0.10600155884645363
Proportion of reports with negative sentiment:  0.8877630553390491


## Sentiment Analysis for Complaint Reports Against Officers with <= 50th TRR Percentile

In [14]:
# query from the DB and store as data frame - complaint reports for officers with <= 50th TRR percentile

query_50 = "SELECT crid as cr_id, cr_text FROM data_allegation a JOIN data_officerallegation d on a.crid = d.allegation_id JOIN data_officer o ON d.officer_id = o.id WHERE trr_percentile <= 50 and ((cr_text <> '' AND cr_text IS NOT NULL) OR summary <> '');"
df_50 = pd.read_sql(query_50, con=connection)

display(df_50)

,cr_id,cr_text
0,1053507,Initial / Intake Allegation 1: The reporting p...
1,1089060,None
2,1036076,None
3,1044943,None
4,1048999,Initial / Intake Allegation 1: The reporting v...
...,...,...
1791,307786,None
1792,1054194,Initial / Intake Allegation 1: The reporting p...
1793,1075825,Initial / Intake Allegation 1: The reporting v...
1794,1002797,None


In [15]:
# split the cr_text column into a column name and text body

df_50[['column_name', 'text']] = df_50['cr_text'].str.split(":", 1, expand = True)

display(df_50)

,cr_id,cr_text,column_name,text
0,1053507,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that an\nunknown ...
1,1089060,None,None,None
2,1036076,None,None,None
3,1044943,None,None,None
4,1048999,Initial / Intake Allegation 1: The reporting v...,Initial / Intake Allegation 1,The reporting victim alleges that the\naccuse...
...,...,...,...,...
1791,307786,None,None,None
1792,1054194,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that he called\nt...
1793,1075825,Initial / Intake Allegation 1: The reporting v...,Initial / Intake Allegation 1,The reporting victim alleged that the\naccuse...
1794,1002797,None,None,None


In [16]:
# remove rows where column name is not 'Initial / Intake Allegation' or 'Allegation'

df_50 = df_50[df_50['column_name'].isin(['Initial / Intake Allegation 1', 'Allegation 1'])]

display(df_50)

,cr_id,cr_text,column_name,text
0,1053507,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that an\nunknown ...
4,1048999,Initial / Intake Allegation 1: The reporting v...,Initial / Intake Allegation 1,The reporting victim alleges that the\naccuse...
5,1049033,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the accused\...
6,1049267,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges the accused\narri...
7,1049267,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges the accused\narri...
...,...,...,...,...
1784,1075113,Initial / Intake Allegation 1: The complainan...,Initial / Intake Allegation 1,The complainant alleges that the accused\nof...
1785,1075504,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party victim alleges that she\n...
1786,1075733,Initial / Intake Allegation 1: The complainant...,Initial / Intake Allegation 1,The complainant alleged the accused officer\n...
1792,1054194,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that he called\nt...


In [17]:
# remove return and new line characters 

df_50['text'] = df_50['text'].str.replace('\\n', ' ')
df_50['text'] = df_50['text'].str.replace('\\r', ' ')

# remove extra whitespace

df_50['text'] = df_50['text'].str.strip()

display(df_50)

/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/338075316.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_50['text'] = df_50['text'].str.replace('\\n', ' ')
/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/338075316.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_50['text'] = df_50['text'].str.replace('\\r', ' ')


,cr_id,cr_text,column_name,text
0,1053507,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that an unknown of...
4,1048999,Initial / Intake Allegation 1: The reporting v...,Initial / Intake Allegation 1,The reporting victim alleges that the accused ...
5,1049033,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that the accused o...
6,1049267,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges the accused arrive...
7,1049267,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges the accused arrive...
...,...,...,...,...
1784,1075113,Initial / Intake Allegation 1: The complainan...,Initial / Intake Allegation 1,The complainant alleges that the accused offic...
1785,1075504,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party victim alleges that she we...
1786,1075733,Initial / Intake Allegation 1: The complainant...,Initial / Intake Allegation 1,The complainant alleged the accused officer fa...
1792,1054194,Initial / Intake Allegation 1: The reporting p...,Initial / Intake Allegation 1,The reporting party alleges that he called the...


In [18]:
# drop unnecessary columns

df_50 = df_50.drop(['cr_text', 'column_name'], axis=1)

display(df_50)

,cr_id,text
0,1053507,The reporting party alleges that an unknown of...
4,1048999,The reporting victim alleges that the accused ...
5,1049033,The reporting party alleges that the accused o...
6,1049267,The reporting party alleges the accused arrive...
7,1049267,The reporting party alleges the accused arrive...
...,...,...
1784,1075113,The complainant alleges that the accused offic...
1785,1075504,The reporting party victim alleges that she we...
1786,1075733,The complainant alleged the accused officer fa...
1792,1054194,The reporting party alleges that he called the...


In [19]:
# remove duplicates

df_50 = df_50.sort_values('text').drop_duplicates('cr_id', keep='last')
df_50 = df_50.sort_values('cr_id')

display(df_50)

,cr_id,text
1255,1048964,The reporting party alleges that he was a vict...
585,1048967,It is alleged that the accused was inattentive...
355,1048977,The reporting party alleges that a male Hispan...
4,1048999,The reporting victim alleges that the accused ...
741,1049002,"baton. The reporting party, WHO DID NOT WITNES..."
...,...,...
657,1075972,It is reported that the accused signed the rad...
1672,1075976,The Reporting Party Victim alleged that the ac...
651,1075985,It is reported that after she gave the complai...
1101,1076006,The reporting victim alleged the accused faile...


In [20]:
# make all text lowercase

df_50['text'] = df_50['text'].str.lower()

display(df_50)

,cr_id,text
1255,1048964,the reporting party alleges that he was a vict...
585,1048967,it is alleged that the accused was inattentive...
355,1048977,the reporting party alleges that a male hispan...
4,1048999,the reporting victim alleges that the accused ...
741,1049002,"baton. the reporting party, who did not witnes..."
...,...,...
657,1075972,it is reported that the accused signed the rad...
1672,1075976,the reporting party victim alleged that the ac...
651,1075985,it is reported that after she gave the complai...
1101,1076006,the reporting victim alleged the accused faile...


In [21]:
# analyze positive and negative sentiment

sentiments_50 = []
positive_50 = 0
negative_50 = 0
neutral_50 = 0

for i in range(df_50.shape[0]):
    scores = sentiment_analyzer.polarity_scores(df_50.iloc[i]['text'])
    sentiments_50.append(scores["compound"])
    if scores["compound"] > 0:
        positive_50 += 1
    elif scores["compound"] < 0:
        negative_50 += 1
    else:
        neutral_50 += 1
        
print("Total reports: ", len(sentiments_50))
print("Number of reports with positive sentiment: ", positive_50)
print("Number of reports with negative sentiment: ", negative_50)
print("Number of reports with neutral sentiment: ", neutral_50)

Total reports:  1105
Number of reports with positive sentiment:  121
Number of reports with negative sentiment:  978
Number of reports with neutral sentiment:  6


In [22]:
# calculate the proportion of positive and negative sentiments

proportion_pos_50 = positive_50 / len(sentiments_50)
print("Proportion of reports with positive sentiment: ", proportion_pos_50)

proportion_neg_50 = negative_50 / len(sentiments_50)
print("Proportion of reports with negative sentiment: ", proportion_neg_50)

Proportion of reports with positive sentiment:  0.10950226244343891
Proportion of reports with negative sentiment:  0.8850678733031674


## Trying an Alternative Method - Training the Entire Dataset Instead of Partitioning First

In [23]:
# query from the DB and store as data frame - all complaint reports

query = "SELECT crid as cr_id, cr_text, trr_percentile FROM data_allegation a JOIN data_officerallegation d on a.crid = d.allegation_id JOIN data_officer o ON d.officer_id = o.id and ((cr_text <> '' AND cr_text IS NOT NULL) OR summary <> '');"
df = pd.read_sql(query, con=connection)

display(df)

,cr_id,cr_text,trr_percentile
0,1053767,Initial / Intake Allegation 1: It is reported ...,49.1036
1,1054738,Initial / Intake Allegation 1: The complainant...,0.0000
2,1058889,Initial / Intake Allegation 1: party\nto go to...,98.4398
3,1074496,Initial / Intake Allegation 1: The complainant...,0.0000
4,1075457,None,84.9030
...,...,...,...
8163,1045777,None,78.2707
8164,1054721,Initial / Intake Allegation 1: The reporting p...,0.0000
8165,1056371,Initial / Intake Allegation 1: The reporting p...,97.9632
8166,1058913,Initial / Intake Allegation 1: It is reported ...,97.0101


In [24]:
# split the cr_text column into a column name and text body

df[['column_name', 'text']] = df['cr_text'].str.split(":", 1, expand = True)

display(df)

,cr_id,cr_text,trr_percentile,column_name,text
0,1053767,Initial / Intake Allegation 1: It is reported ...,49.1036,Initial / Intake Allegation 1,It is reported that the accused officers\nsub...
1,1054738,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,"The complainant alleges that her son,\nand a ..."
2,1058889,Initial / Intake Allegation 1: party\nto go to...,98.4398,Initial / Intake Allegation 1,"party\nto go to court on\nof radio\nagencey,\..."
3,1074496,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,The complainant alleged that the accused\npol...
4,1075457,None,84.9030,None,None
...,...,...,...,...,...
8163,1045777,None,78.2707,None,None
8164,1054721,Initial / Intake Allegation 1: The reporting p...,0.0000,Initial / Intake Allegation 1,The reporting party alleged that she and the\...
8165,1056371,Initial / Intake Allegation 1: The reporting p...,97.9632,Initial / Intake Allegation 1,The reporting party further alleged that\nwhi...
8166,1058913,Initial / Intake Allegation 1: It is reported ...,97.0101,Initial / Intake Allegation 1,It is reported that while the accused was.\ni...


In [25]:
# remove rows where column name is not 'Initial / Intake Allegation' or 'Allegation'

df = df[df['column_name'].isin(['Initial / Intake Allegation 1', 'Allegation 1'])]

display(df)

,cr_id,cr_text,trr_percentile,column_name,text
0,1053767,Initial / Intake Allegation 1: It is reported ...,49.1036,Initial / Intake Allegation 1,It is reported that the accused officers\nsub...
1,1054738,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,"The complainant alleges that her son,\nand a ..."
2,1058889,Initial / Intake Allegation 1: party\nto go to...,98.4398,Initial / Intake Allegation 1,"party\nto go to court on\nof radio\nagencey,\..."
3,1074496,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,The complainant alleged that the accused\npol...
59,1049257,Initial / Intake Allegation 1: The reporting p...,77.7091,Initial / Intake Allegation 1,The reporting party alleges that the accused\...
...,...,...,...,...,...
8139,1075825,Initial / Intake Allegation 1: The reporting v...,38.9028,Initial / Intake Allegation 1,The reporting victim alleged that the\naccuse...
8164,1054721,Initial / Intake Allegation 1: The reporting p...,0.0000,Initial / Intake Allegation 1,The reporting party alleged that she and the\...
8165,1056371,Initial / Intake Allegation 1: The reporting p...,97.9632,Initial / Intake Allegation 1,The reporting party further alleged that\nwhi...
8166,1058913,Initial / Intake Allegation 1: It is reported ...,97.0101,Initial / Intake Allegation 1,It is reported that while the accused was.\ni...


In [26]:
# remove return and new line characters 

df['text'] = df['text'].str.replace('\\n', ' ')
df['text'] = df['text'].str.replace('\\r', ' ')

# remove extra whitespace

df['text'] = df['text'].str.strip()

display(df)

/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/334825093.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\\n', ' ')
/var/folders/ks/ydq3x3c119x8p1vc_m_vp0rm0000gn/T/ipykernel_99663/334825093.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\\r', ' ')


,cr_id,cr_text,trr_percentile,column_name,text
0,1053767,Initial / Intake Allegation 1: It is reported ...,49.1036,Initial / Intake Allegation 1,It is reported that the accused officers submi...
1,1054738,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,"The complainant alleges that her son, and a bl..."
2,1058889,Initial / Intake Allegation 1: party\nto go to...,98.4398,Initial / Intake Allegation 1,"party to go to court on of radio agencey, Depa..."
3,1074496,Initial / Intake Allegation 1: The complainant...,0.0000,Initial / Intake Allegation 1,The complainant alleged that the accused polic...
59,1049257,Initial / Intake Allegation 1: The reporting p...,77.7091,Initial / Intake Allegation 1,The reporting party alleges that the accused o...
...,...,...,...,...,...
8139,1075825,Initial / Intake Allegation 1: The reporting v...,38.9028,Initial / Intake Allegation 1,The reporting victim alleged that the accused ...
8164,1054721,Initial / Intake Allegation 1: The reporting p...,0.0000,Initial / Intake Allegation 1,The reporting party alleged that she and the a...
8165,1056371,Initial / Intake Allegation 1: The reporting p...,97.9632,Initial / Intake Allegation 1,The reporting party further alleged that while...
8166,1058913,Initial / Intake Allegation 1: It is reported ...,97.0101,Initial / Intake Allegation 1,It is reported that while the accused was. ins...


In [27]:
# drop unnecessary columns

df = df.drop(['cr_text', 'column_name'], axis=1)

display(df)

,cr_id,trr_percentile,text
0,1053767,49.1036,It is reported that the accused officers submi...
1,1054738,0.0000,"The complainant alleges that her son, and a bl..."
2,1058889,98.4398,"party to go to court on of radio agencey, Depa..."
3,1074496,0.0000,The complainant alleged that the accused polic...
59,1049257,77.7091,The reporting party alleges that the accused o...
...,...,...,...
8139,1075825,38.9028,The reporting victim alleged that the accused ...
8164,1054721,0.0000,The reporting party alleged that she and the a...
8165,1056371,97.9632,The reporting party further alleged that while...
8166,1058913,97.0101,It is reported that while the accused was. ins...


In [28]:
# remove duplicates

df = df.sort_values('text').drop_duplicates('cr_id', keep='last')
df = df.sort_values('cr_id')

display(df)

,cr_id,trr_percentile,text
1238,1048962,81.0961,The victim alleges that an unknown male black ...
6385,1048964,0.0000,The reporting party alleges that he was a vict...
1239,1048966,66.4142,The Reporting Party Victim stated that she tel...
4434,1048967,0.0000,It is alleged that the accused was inattentive...
3392,1048976,72.1094,The complainant alleges that the accused faile...
...,...,...,...
3182,1076026,99.4610,The reporting victim alleges that the accused ...
8097,1076050,0.0000,The reporting party alleged that the accused w...
5222,1076159,90.8246,The reporting party/victim alleg t the accused...
4193,1076168,92.8571,The complainant related the accused officer ar...


In [29]:
# make all text lowercase

df['text'] = df['text'].str.lower()

display(df)

,cr_id,trr_percentile,text
1238,1048962,81.0961,the victim alleges that an unknown male black ...
6385,1048964,0.0000,the reporting party alleges that he was a vict...
1239,1048966,66.4142,the reporting party victim stated that she tel...
4434,1048967,0.0000,it is alleged that the accused was inattentive...
3392,1048976,72.1094,the complainant alleges that the accused faile...
...,...,...,...
3182,1076026,99.4610,the reporting victim alleges that the accused ...
8097,1076050,0.0000,the reporting party alleged that the accused w...
5222,1076159,90.8246,the reporting party/victim alleg t the accused...
4193,1076168,92.8571,the complainant related the accused officer ar...


In [30]:
# analyze positive and negative sentiment among all reports

sentiments = []
positive = 0
negative = 0
neutral = 0

for i in range(df.shape[0]):
    scores = sentiment_analyzer.polarity_scores(df.iloc[i]['text'])
    sentiments.append(scores["compound"])
    if scores["compound"] > 0:
        positive += 1
    elif scores["compound"] < 0:
        negative += 1
    else:
        neutral += 1
        
print("Total reports: ", len(sentiments))
print("Number of reports with positive sentiment: ", positive)
print("Number of reports with negative sentiment: ", negative)
print("Number of reports with neutral sentiment: ", neutral)

Total reports:  3391
Number of reports with positive sentiment:  352
Number of reports with negative sentiment:  3019
Number of reports with neutral sentiment:  20


In [31]:
# calculate the proportion of positive and negative sentiments

proportion_pos = positive / len(sentiments)
print("Proportion of reports with positive sentiment: ", proportion_pos)

proportion_neg = negative / len(sentiments)
print("Proportion of reports with negative sentiment: ", proportion_neg)

Proportion of reports with positive sentiment:  0.10380418755529343
Proportion of reports with negative sentiment:  0.8902978472427012


In [32]:
# analyze positive and negative sentiment then partition into strata

sentiments_new = []

positive_90_2 = 0
negative_90_2 = 0
neutral_90_2 = 0

positive_50_2 = 0
negative_50_2 = 0
neutral_50_2 = 0

for i in range(df.shape[0]):
    scores = sentiment_analyzer.polarity_scores(df.iloc[i]['text'])
    sentiments_new.append(scores["compound"])
    if scores["compound"] > 0:
        if df.iloc[i]['trr_percentile'] >= 90: 
            positive_90_2 += 1
        if df.iloc[i]['trr_percentile'] <= 50: 
            positive_50_2 += 1
    elif scores["compound"] < 0:
        if df.iloc[i]['trr_percentile'] >= 90: 
            negative_90_2 += 1
        if df.iloc[i]['trr_percentile'] <= 50: 
            negative_50_2 += 1
    else:
        if df.iloc[i]['trr_percentile'] >= 90: 
            neutral_90_2 += 1
        if df.iloc[i]['trr_percentile'] <= 50: 
            neutral_50_2 += 1
        
print("Total reports: ", len(sentiments_new))
print("Number of reports with positive sentiment against officers in the 90th percentile or above: ", positive_90_2)
print("Number of reports with negative sentiment against officers in the 90th percentile or above: ", negative_90_2)
print("Number of reports with neutral sentiment against officers in the 90th percentile or above: ", neutral_90_2)
print("Number of reports with positive sentiment against officers in the 50th percentile or below: ", positive_50_2)
print("Number of reports with negative sentiment against officers in the 50th percentile or below: ", negative_50_2)
print("Number of reports with neutral sentiment against officers in the 50th percentile or below: ", neutral_50_2)

Total reports:  3391
Number of reports with positive sentiment against officers in the 90th percentile or above:  99
Number of reports with negative sentiment against officers in the 90th percentile or above:  827
Number of reports with neutral sentiment against officers in the 90th percentile or above:  8
Number of reports with positive sentiment against officers in the 50th percentile or below:  95
Number of reports with negative sentiment against officers in the 50th percentile or below:  726
Number of reports with neutral sentiment against officers in the 50th percentile or below:  4


In [33]:
# calculate the proportion of positive and negative sentiments against officers in the 90th percentile or above

total_90 = positive_90_2 + negative_90_2 + neutral_90_2

proportion_pos_90_2 = positive_90_2 / total_90
print("Proportion of reports with positive sentiment against officers in the 90th percentile or above: ", proportion_pos_90_2)

proportion_neg_90_2 = negative_90_2 / total_90
print("Proportion of reports with negative sentiment against officers in the 90th percentile or above: ", proportion_neg_90_2)

Proportion of reports with positive sentiment against officers in the 90th percentile or above:  0.10599571734475374
Proportion of reports with negative sentiment against officers in the 90th percentile or above:  0.8854389721627409


In [34]:
# calculate the proportion of positive and negative sentiments against officers in the 50th percentile or below

total_50 = positive_50_2 + negative_50_2 + neutral_50_2

proportion_pos_50_2 = positive_50_2 / total_50
print("Proportion of reports with positive sentiment against officers in the 50th percentile or below: ", proportion_pos_50_2)

proportion_neg_50_2 = negative_50_2 / total_50
print("Proportion of reports with negative sentiment against officers in the 50th percentile or below: ", proportion_neg_50_2)

Proportion of reports with positive sentiment against officers in the 50th percentile or below:  0.11515151515151516
Proportion of reports with negative sentiment against officers in the 50th percentile or below:  0.88


## Evaluating the Accuracy of the NLTK Model

In [44]:
# select 30 CRs to use as test data

test_df = df.head(n=30)

display(test_df)

,cr_id,trr_percentile,text
1238,1048962,81.0961,the victim alleges that an unknown male black ...
6385,1048964,0.0000,the reporting party alleges that he was a vict...
1239,1048966,66.4142,the reporting party victim stated that she tel...
4434,1048967,0.0000,it is alleged that the accused was inattentive...
3392,1048976,72.1094,the complainant alleges that the accused faile...
2288,1048977,0.0000,the reporting party alleges that a male hispan...
1241,1048978,95.5066,the reporting party alleges that several unifo...
6386,1048979,97.1463,the complainant alleges that the accused appro...
134,1048986,64.4078,it is reported that the accused was observed d...
135,1048989,81.0961,the complainant alleges that the accused arres...


In [45]:
# print each CR and whether the model classifies it as positive, negative, or neutral

sentiments_test = []
positive_test = 0
negative_test = 0
neutral_test = 0

for i in range(test_df.shape[0]):
    scores = sentiment_analyzer.polarity_scores(test_df.iloc[i]['text'])
    sentiments_test.append(scores["compound"])
    if scores["compound"] > 0:
        print(test_df.iloc[i]['text'], " - positive\n\n")
        positive_test += 1
    elif scores["compound"] < 0:
        print(test_df.iloc[i]['text'], " - negative\n\n")
        negative_test += 1
    else:
        print(test_df.iloc[i]['text'], " - neutral\n\n")
        neutral_test += 1

the victim alleges that an unknown male black off-duty officer threw her against the wall after she did not adhere to his command to get up. the victim alleges that the officer pushed her face against the wall and handcuffed her too tightly. allegation 3: itis alleged that the accused threw bushing her face againsta wall it is alleged that the accused threw n the floor. itis alleged that .~ ras handcuffed too tight. itis alleged that the accused dragged (- the stairs. itis alleged that the acoused grabbed [ lom behind her neck finding 1: unfounded  - negative


the reporting party alleges that he was a victim of a battery and when the police arrived they failed to file a report on his behalf instead, the officers gave him a ride to his friend's house. finding 1: (none entered)  - negative


the reporting party victim stated that she telephoned “311" regarding recovery of her stolen vehicle. the victim alleged that the responding officer never took her vehicle off the hot spot sheet whi